In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym
import numpy as np

# Define Policy Network
class Policy(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# Discounted rewards function
def discount_rewards(rewards, gamma=0.99):
    discounted_rewards = []
    running_add = 0
    for r in reversed(rewards):
        running_add = running_add * gamma + r
        discounted_rewards.insert(0, running_add)
    return discounted_rewards

# Initialize the environment
env = gym.make('CartPole-v1')
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

# Initialize the policy network
policy = Policy(input_dim, output_dim)

# Initialize the optimizer
optimizer = optim.Adam(policy.parameters(), lr=0.01)

# Training loop
for episode in range(1000):
    state = env.reset()
    episode_reward = 0

    states = []
    actions = []
    rewards = []

    while True:
        env.render()  # Render the environment

        states.append(torch.tensor(state, dtype=torch.float32))

        action_probs = policy(torch.tensor(state, dtype=torch.float32))
        action = torch.multinomial(action_probs, num_samples=1).item()
        actions.append(action)

        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)

        episode_reward += reward
        state = next_state

        if done:
            # Compute discounted rewards
            discounted_rewards = torch.tensor(discount_rewards(rewards), dtype=torch.float32)
            
            # Compute policy loss
            log_probs = torch.log(policy(torch.stack(states)))[range(len(actions)), actions]
            policy_loss = -(log_probs * discounted_rewards).mean()

            # Update policy
            optimizer.zero_grad()
            policy_loss.backward()
            optimizer.step()

            print("Episode: {}, Reward: {}".format(episode, episode_reward))
            break

env.close()


Episode: 27, Reward: 42.0
Episode: 28, Reward: 16.0
Episode: 29, Reward: 20.0
Episode: 30, Reward: 27.0
Episode: 31, Reward: 18.0
Episode: 32, Reward: 33.0
Episode: 33, Reward: 23.0
Episode: 34, Reward: 21.0
Episode: 35, Reward: 37.0
Episode: 36, Reward: 46.0
Episode: 37, Reward: 13.0
Episode: 38, Reward: 16.0
Episode: 39, Reward: 16.0
Episode: 40, Reward: 22.0
Episode: 41, Reward: 20.0
Episode: 42, Reward: 21.0
Episode: 43, Reward: 13.0
Episode: 44, Reward: 37.0
Episode: 45, Reward: 21.0
Episode: 46, Reward: 40.0
Episode: 47, Reward: 15.0
Episode: 48, Reward: 19.0
Episode: 49, Reward: 10.0
Episode: 50, Reward: 22.0
Episode: 51, Reward: 25.0
Episode: 52, Reward: 35.0
